# Table 1: Regressive vs Progressive MSA

This notebook contains all the code to generate Table 1 from the publication.

### _Input Data_
Our input data consists of 94 datasets from the HOMFAM benchmark. Each of ther datasets is a protein family with the number sequences in each ranging from 88 for the smallest family *seatoxin* to 93,675 for the largest family *rvp*.

In [6]:
cat ../data/seqs/seatoxin.fa | grep '>'| wc -l

      88


In [7]:
cat ../data/seqs/rvp.fa | grep '>'| wc -l

   93675


For each dataset, there is also a reference set of sequences.
These references are from the PDB and have been structually aligned.

In [10]:
cat ../data/refs/seatoxin.ref

>1apf
GVPCLCDSDGPRPRGNTLSGILWFYPSGCPS--GWH-NCKAHGPNIGWCCKK--
>1ahl
GVSCLCDSDGPSVRGNTLSGTLWLYPSGCPS--GWH-NCKAHGPTIGWCCKQ--
>1atx
GAACLCKSDGPNTRGNSMSGTIWVF--GCPS--GWN-NCEGRA-IIGYCCKQ--
>1sh1
-AACKCDDEGPDIRTAPLTGTVDLG--SCNA--GWE-KCASYYTIIADCCRKKK
>1bds
AAPCFCSGKP-------GRGDLWILRGTCPGGYGYTSNCYK--WPNICCYPH--


Each sequence file has been combined with the unaligned reference sequences to create a combined sequence set.

In [16]:
!head ../data/combined_seqs/seatoxin.fa

>B1NWT1_NEMVE/41-81
ACACDSPGIRSASLSGIVWVGSCPSGWKKCKSYYSVVADCC
>B1NWR7_NEMVE/41-83
PCACDSDGPDIRSASLSGIVWMGSCPSGWKKCKSYYSIVADCC
>TXCN2_BUNCN/3-46
ACRCDSDGPTVRGDSLSGTLWLTGGCPSGWHNCRGSGPFIGYCC
>TX5_ANTXA/3-45
SCLCDSDGPSVSGNTLSGIIWLAGCPSGWHNCKAHGPNIGWCC
>TXH7_ANTS7/3-45
PCLCDSDGPSVHGNTLSGTIWLAGCPSGWHNCKAHGPTIGWCC


### _Command lines used in workflow to generate guide trees_

#### Clustal Omega Trees (mBed)
```
custalo -i ${seqs} --guidetree-out ${id}.${clustalo}.dnd
```

#### MAFFT PartTree trees
```
t_coffee -other_pg seq_reformat                 \
          -in ${seqs} -action +seq2dnd parttree \
          -output newick                        \
          >> ${id}.MAFFT_PARTTREE.dnd
```               

### _Command lines used in workflow to generate alignments_

#### Clustal Omega progressive alignments
```
custalo -i ${seqs} --guidetree-out ${id}.${clustalo}.dnd
```

#### MAFFT-FFTNS1 progressive alignments (3 commands)
```
t_coffee -other_pg seq_reformat \
            -in ${guide_tree} -input newick \
            -in2 ${seqs} -input2 fasta_seq  \
            -action +newick2mafftnewick     \
            >> ${id}.mafftnewick

newick2mafft.rb 1.0 ${id}.mafftnewick > ${id}.mafftbinary

mafft --retree 1 --anysymbol              \
       --treein ${id}.mafftbinary ${seqs} \
       > ${id}.std.MAFFT-FFTNS1.with.${tree_method}.tree.aln
```

#### Clustal Omega regressive (dpa) alignments
```
t_coffee -dpa -dpa_method clustalo_msa \
         -dpa_tree ${guide_tree}       \
         -seq ${seqs}                  \
         -dpa_nseq ${bucket_size}      \
         -outfile ${id}.dpa_${bucket_size}.CLUSTALO.with.${tree_method}.tree.aln
```

#### MAFFT-FFTNS1 regressive (dpa) alignments
```
t_coffee -dpa -dpa_method mafftfftns1_msa \
         -dpa_tree ${guide_tree} \
         -seq ${seqs} \
         -dpa_nseq ${bucket_size} \
         -outfile ${id}.dpa_${bucket_size}.CLUSTALO.with.${tree_method}.tree.aln
```

### Workflow

The main workflow for generating the alignments is written in Nextflow (http://nextflow.io)


### Part One: Progressive vs Regressive Alignments
The first table compares __Progressive vs Regressive__ alignment proceedurees using two of the most common large scale tree building and alignment proceedures.
* Clustal Omega with Clustal Omega trees
* Clustal Omega with MAFFT PartTree trees
* MAFFT FFT-NS-1 with MAFFT PartTree trees
* MAFFT FFT-NS-1 with Clustal Omega trees

The command to run for running the workflow is as follows:

In [ ]:
!~/bin/nextflow run ../main.nf \
                    --align_method="CLUSTALO,MAFFT-FFTNS1" \
                    --tree_method="CLUSTALO,MAFFT_PARTTREE" \
                    --refs='../data/refs/*.ref' \
                    --seqs='../data/combined_seqs/*.fa' \
                    --dpa_align \
                    --std_align \
                    --default_align=false \
                    --output results/publication_table_1a \
                    -with-docker \
                    -resume

#### Import required python libraries

In [2]:
import plotly.plotly as py
import plotly.figure_factory as ff
import os
import csv

#### Create a dictionary for each familiy with values being the number of sequences in the dataset.

In [3]:
with open("../data/num_seqs.csv", mode='r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    sizes_dict = {rows[0]:rows[1] for rows in reader}

#### Create a nested dictionary containing all the scores from the individual scores files

In [8]:
all_files = []
score_dict = {}

individual_scores_dir="results/publication_table_1a/individual_scores/"
for filename in os.listdir(individual_scores_dir):
     # seatoxin.std_align.NA.MAFFT-FFTNS1.MAFFT_PARTTREE.col
     family, align_type, bucket, aligner, tree, score_type = filename.split('.')
     y = [align_type, aligner, tree, family, score_type]
     with open(individual_scores_dir+filename, 'r') as myfile:
         data = myfile.read()
     y.append(data.rstrip())
     all_files.append(y)
        
for filename in all_files:
    current_level = score_dict
    for part in filename:
        if part not in current_level:
            current_level[part] = {}
        current_level = current_level[part]

#### Calculate the average total coloumn score for families containing  > 10,000 seqs

In [18]:
tc_scores_dict={}
for k, v in score_dict.items():   
    for k1, v1 in v.items():
        for k2, v2 in v1.items():
            n=0
            sum_dict = {'sp':0, 'tc':0, 'col':0 }
            for k3, v3 in v2.items():
                if int(sizes_dict[k3]) > 10000:               
                    n+=1
                    for k4, v4 in v3.items():
                        for k5, v5 in v4.items():
                            sum_dict[k4]+=float(k5)
            tc_avg = round((sum_dict['tc']/n), 2)
            key=(k+k1+k2)
            tc_scores_dict[key] = tc_avg
print("Read in",n,"scores")

Read in 20 scores


#### Generate the table

In [21]:
alignment_methods=['CLUSTALO','MAFFT-FFTNS1']
tree_methods=['CLUSTALO', 'MAFFT_PARTTREE']

rows=[['Alignment Methdod', 'Tree Method', 'Progressive', 'Regressive']]
for a_method in alignment_methods:
    for t_method in tree_methods:
        rows.append([a_method,
                     t_method, 
                     tc_scores_dict["std_align"+a_method+t_method], 
                     tc_scores_dict["dpa_align"+a_method+t_method]])

table = ff.create_table(rows)
py.iplot(table, filename='table1_a')